## Pytorch

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

## Fairseq

In [2]:
!git clone https://github.com/pytorch/fairseq.git

Cloning into 'fairseq'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 2778 (delta 42), reused 59 (delta 33), pack-reused 2698
Receiving objects: 100% (2778/2778), 2.99 MiB | 2.69 MiB/s, done.
Resolving deltas: 100% (2007/2007), done.


In [0]:
import os
os.chdir("fairseq/")

In [4]:
!pip install -r requirements.txt
%run -i 'setup.py' build develop

    100% |████████████████████████████████| 430kB 7.2MB/s 
    100% |████████████████████████████████| 163kB 9.4MB/s 
  Running setup.py bdist_wheel for pycparser ... - \ | done
  Stored in directory: /root/.cache/pip/wheels/f2/9a/90/de94f8556265ddc9d9c8b271b0f63e57b26fb1d67a45564511
Successfully built pycparser
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.6
creating build/lib.linux-x86_64-3.6/fairseq
copying fairseq/tokenizer.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/distributed_utils.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/trainer.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/meters.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/__init__.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/multiprocessing_pdb.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/progress_bar.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/sequence_scorer.py -> build/lib.linux-x86_64-3.6/fairs

## Fairseq Tutorials

### Evaluting Pretrained models

In [5]:
!curl https://s3.amazonaws.com/fairseq-py/models/wmt14.v2.en-fr.fconv-py.tar.bz2 | tar xvjf -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0 1909M    0  610k    0     0   327k      0  1:39:38  0:00:01  1:39:37  326kwmt14.en-fr.fconv-py/
wmt14.en-fr.fconv-py/model.pt
 99 1909M   99 1908M    0     0  3585k      0  0:09:05  0:09:05 --:--:-- 3495kwmt14.en-fr.fconv-py/dict.en.txt
wmt14.en-fr.fconv-py/dict.fr.txt
100 1909M  100 1909M    0     0  3585k      0  0:09:05  0:09:05 --:--:-- 3443k
wmt14.en-fr.fconv-py/bpecodes
wmt14.en-fr.fconv-py/README.md


In [6]:
MODEL_DIR="wmt14.en-fr.fconv-py"
%run -i 'interactive.py' --path $MODEL_DIR/model.pt $MODEL_DIR --beam 5 --source-lang en --target-lang fr
#Dont know how to run interactive in notebooks

Namespace(beam=5, buffer_size=1, cpu=False, data=['wmt14.en-fr.fconv-py'], diverse_beam_groups=1, diverse_beam_strength=0.5, fp16=False, fp16_init_scale=128, gen_subset='test', left_pad_source='True', left_pad_target='False', lenpen=1, log_format=None, log_interval=1000, max_len_a=0, max_len_b=200, max_sentences=1, max_source_positions=1024, max_target_positions=1024, max_tokens=None, min_len=1, model_overrides='{}', nbest=1, no_beamable_mm=False, no_early_stop=False, no_progress_bar=False, num_shards=1, path='wmt14.en-fr.fconv-py/model.pt', prefix_size=0, print_alignment=False, quiet=False, raw_text=False, remove_bpe=None, replace_unk=None, sampling=False, sampling_temperature=1, sampling_topk=-1, score_reference=False, seed=1, shard_id=0, skip_invalid_size_inputs_valid_test=False, source_lang='en', target_lang='fr', task='translation', unkpen=0, unnormalized=False, upsample_primary=1)
| [en] dictionary: 43771 types
| [fr] dictionary: 43807 types
| loading model(s) from wmt14.en-fr.fc

In [7]:
!curl https://s3.amazonaws.com/fairseq-py/data/wmt14.v2.en-fr.newstest2014.tar.bz2 | tar xvjf -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  2  676k    2 17039    0     0  16853      0  0:00:41  0:00:01  0:00:40 16853wmt14.en-fr.newstest2014/
wmt14.en-fr.newstest2014/test.en-fr.en.bin
wmt14.en-fr.newstest2014/test.en-fr.en.idx
wmt14.en-fr.newstest2014/test.en-fr.fr.bin
wmt14.en-fr.newstest2014/test.en-fr.fr.idx
wmt14.en-fr.newstest2014/dict.en.txt
100  676k  100  676k    0     0   385k      0  0:00:01  0:00:01 --:--:--  385k
wmt14.en-fr.newstest2014/dict.fr.txt


In [11]:
output = %run 'generate.py' wmt14.en-fr.newstest2014 --path $MODEL_DIR/model.pt --beam 5 --batch-size 128 --remove-bpe

Namespace(beam=5, cpu=False, data=['wmt14.en-fr.newstest2014'], diverse_beam_groups=1, diverse_beam_strength=0.5, fp16=False, fp16_init_scale=128, gen_subset='test', left_pad_source='True', left_pad_target='False', lenpen=1, log_format=None, log_interval=1000, max_len_a=0, max_len_b=200, max_sentences=128, max_source_positions=1024, max_target_positions=1024, max_tokens=None, min_len=1, model_overrides='{}', nbest=1, no_beamable_mm=False, no_early_stop=False, no_progress_bar=False, num_shards=1, path='wmt14.en-fr.fconv-py/model.pt', prefix_size=0, print_alignment=False, quiet=False, raw_text=False, remove_bpe='@@ ', replace_unk=None, sampling=False, sampling_temperature=1, sampling_topk=-1, score_reference=False, seed=1, shard_id=0, skip_invalid_size_inputs_valid_test=False, source_lang=None, target_lang=None, task='translation', unkpen=0, unnormalized=False, upsample_primary=1)
| [en] dictionary: 43771 types
| [fr] dictionary: 43807 types
| wmt14.en-fr.newstest2014 test 3003 examples


In [12]:
with open('gen.txt', 'w') as f:
    f.write(output)

TypeError: ignored